In [5]:
import pandas as pd

In [6]:
trips_df = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    'client_id': [1, 2, 3, 4, 1, 2, 3, 2, 3, 4],
    'driver_id': [10, 11, 12, 13, 10, 11, 12, 12, 10, 13],
    'city_id': [1, 1, 6, 6, 1, 6, 6, 12, 12, 12],
    'status': ['completed', 'cancelled_by_driver', 'completed', 'cancelled_by_client', 'completed', 'completed', 'completed', 'completed', 'completed', 'cancelled_by_driver'],
    'request_at': ['2013-10-01', '2013-10-01', '2013-10-01', '2013-10-01', '2013-10-02', '2013-10-02', '2013-10-02', '2013-10-03', '2013-10-03', '2013-10-03']
})
trips_df

,id,client_id,driver_id,city_id,status,request_at
0,1,1,10,1,completed,2013-10-01
1,2,2,11,1,cancelled_by_driver,2013-10-01
2,3,3,12,6,completed,2013-10-01
3,4,4,13,6,cancelled_by_client,2013-10-01
4,5,1,10,1,completed,2013-10-02
5,6,2,11,6,completed,2013-10-02
6,7,3,12,6,completed,2013-10-02
7,8,2,12,12,completed,2013-10-03
8,9,3,10,12,completed,2013-10-03
9,10,4,13,12,cancelled_by_driver,2013-10-03


In [7]:
users_df = pd.DataFrame({
    'users_id': [1, 2, 3, 4, 10, 11, 12, 13],
    'banned': [False, True, False, False, False, False, False, False],
    'role': ['client', 'client', 'client', 'client', 'driver', 'driver', 'driver', 'driver']
})
users_df

,users_id,banned,role
0,1,False,client
1,2,True,client
2,3,False,client
3,4,False,client
4,10,False,driver
5,11,False,driver
6,12,False,driver
7,13,False,driver


How to calculate the cancellation rate of requests with unbanned users (client and driver both not banned) each day between October 1, 2013, and October 3, 2013? Ensure the solution rounds the cancellation rate to two decimal points.

In [8]:
output = pd.DataFrame({
    "Day": ["2013-10-01", "2013-10-02", "2013-10-03"],
    "Cancellation Rate": [0.33, 0.00, 0.50]
})
output

,Day,Cancellation Rate
0,2013-10-01,0.33
1,2013-10-02,0.00
2,2013-10-03,0.50


Explanation:

On 2013-10-01: Out of 4 total requests, 2 were canceled. Discounting a request from a banned client, there were 3 unbanned requests, with a cancellation rate of 0.33.

On 2013-10-02: Among 3 total requests, none were canceled. Ignoring a request from a banned client, there were 2 unbanned requests, resulting in a cancellation rate of 0.00.

On 2013-10-03: Out of 3 total requests, 1 was canceled. Disregarding a request from a banned client, there were 2 unbanned requests, leading to a cancellation rate of 0.50.

In [108]:
df=trips_df.merge(users_df,how='inner',left_on=['client_id'],right_on=['users_id'])

In [109]:
df.drop('role',axis=1,inplace=True)

In [110]:
df['banned'] = joined_df['banned'].apply(lambda x : 1 if x == True else 0)

In [111]:
df

,id,client_id,driver_id,city_id,status,request_at,users_id,banned
0,1,1,10,1,completed,2013-10-01,1,0
1,5,1,10,1,completed,2013-10-02,1,0
2,2,2,11,1,cancelled_by_driver,2013-10-01,2,1
3,6,2,11,6,completed,2013-10-02,2,1
4,8,2,12,12,completed,2013-10-03,2,1
5,3,3,12,6,completed,2013-10-01,3,0
6,7,3,12,6,completed,2013-10-02,3,0
7,9,3,10,12,completed,2013-10-03,3,0
8,4,4,13,6,cancelled_by_client,2013-10-01,4,0
9,10,4,13,12,cancelled_by_driver,2013-10-03,4,0


In [127]:
(df.groupby(['request_at','status','banned']).agg(requests=('status', 'count')))

requests
request_at status              banned          
2013-10-01 cancelled_by_client 0              1
           cancelled_by_driver 1              1
           completed           0              2
2013-10-02 completed           0              2
                               1              1
2013-10-03 cancelled_by_driver 0              1
           completed           0              1
                               1              1

In [128]:
df_without_banned_canceles = df[~((joined_df['status']!= 'completed')&(joined_df['banned']==1))]

In [129]:
unbanned_df = df_without_banned_canceles[df_without_banned_canceles['banned'] == 0]

In [135]:
canceled_df = df_without_banned_canceles[~(df_without_banned_canceles['status']=='completed')]

In [151]:
final_rate = pd.DataFrame((canceled_df.groupby(['request_at']).count()/unbanned_df.groupby(['request_at']).count()).iloc[:,1])

In [153]:
final_rate = round(final_rate,2)

In [155]:
final_rate.fillna(0)

,client_id
request_at,
2013-10-01,0.33
2013-10-02,0.00
2013-10-03,0.50
